In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import gc
import pickle
import logging
import multiprocessing
from pathlib import Path

import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp
import numpyro

from hbmep.config import Config
from hbmep.model.utils import Site as site

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)

# FORMAT = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
# formatter = logging.Formatter(FORMAT)

# fhandler = logging.FileHandler(filename='power-analysis-parallel.log', mode="w")
# fhandler.setFormatter(formatter)
# fhandler.setLevel(logging.INFO)
# logger.addHandler(fhandler)

# consoleHandler = logging.StreamHandler(sys.stdout)
# consoleHandler.setFormatter(formatter)
# logger.addHandler(consoleHandler)


In [ ]:
from typing import Optional

import arviz as az
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns


def _plot(
    self,
    df: pd.DataFrame,
    dfs: list[pd.DataFrame],
    destination_path: str,
    threshold_dfs: Optional[pd.DataFrame] = None
):
    """ Setup pdf layout """
    combinations = self._make_combinations(df=dfs[0], columns=self.combination_columns)
    n_combinations = len(combinations)

    n_columns_per_response = 1 + len(dfs)

    n_fig_rows = 10
    n_fig_columns = n_columns_per_response * self.n_response

    n_pdf_pages = n_combinations // n_fig_rows
    if n_combinations % n_fig_rows: n_pdf_pages += 1
    logger.info("Rendering ...")

    """ Iterate over pdf pages """
    pdf = PdfPages(destination_path)
    combination_counter = 0

    for page in range(n_pdf_pages):
        n_rows_current_page = min(
            n_fig_rows,
            n_combinations - page * n_fig_rows
        )

        fig, axes = plt.subplots(
            n_rows_current_page,
            n_fig_columns,
            figsize=(
                n_fig_columns * self.subplot_cell_width,
                n_rows_current_page * self.subplot_cell_height
            ),
            constrained_layout=True,
            squeeze=False
        )

        """ Iterate over combinations """
        for i in range(n_rows_current_page):
            curr_combination = combinations[combination_counter]

            """ Filter dataframe based on current combination """
            df_ind = df[self.combination_columns].apply(tuple, axis=1).isin([curr_combination])
            curr_df = df[df_ind].reset_index(drop=True).copy()

            # th_ind = threshold_df[self.combination_columns].apply(tuple, axis=1).isin([curr_combination])
            # curr_th_df = threshold_df[th_ind].reset_index(drop=True).copy()
            # curr_ths = curr_th_df[self.response].values[0]

            """ Iterate over responses """
            j = 0
            for r, response in enumerate(self.response):
                """ Plots: Scatter Plot """
                if curr_df.shape[0]:
                    ax = axes[i, j]
                    sns.scatterplot(data=curr_df, x=self.intensity, y=response, color=self.response_colors[r], ax=ax)

                    if j == 1:
                        ax.legend(loc="upper right")

                    ax.set_title("Real Data - " + response + f" - {curr_combination}")

                j += 1

                for sim_ind, curr_sim in enumerate(dfs):
                    curr_sim = curr_sim.copy()
                    curr_sim_ind = curr_sim[self.combination_columns].apply(tuple, axis=1).isin([curr_combination])
                    curr_sim = curr_sim[curr_sim_ind].reset_index(drop=True).copy()

                    ax = axes[i, j]
                    sns.scatterplot(data=curr_sim, x=self.intensity, y=response, color=self.response_colors[r], ax=ax)
                    # ax.axvline(curr_ths[r], color="black", label="True Threshold")
                    # ax.set_title(f"{N} Pulses, {m} Reps")

                    # if not sim_ind:
                    #     ax.legend(loc="upper right")

                    ax.set_title(f"Simulated: {curr_combination}")

                    j += 1

            combination_counter += 1

        pdf.savefig(fig)
        plt.close()

    pdf.close()
    plt.show()

    logger.info(f"Saved to {destination_path}")
    return




In [ ]:
import numpyro.distributions as dist
from hbmep.model import Baseline
from hbmep_paper.utils.constants import HBM


class Simulation(Baseline):
    LINK = HBM

    def __init__(self, config: Config, mu_a_delta: int, sigma_a_delta: int):
        super(Simulation, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]
        self.mu_a_delta = mu_a_delta
        self.sigma_a_delta = sigma_a_delta

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        n_baseline = 1
        n_delta = 1

        global_sigma_b_baseline = numpyro.sample("global_sigma_b_baseline", dist.HalfNormal(100))
        global_sigma_v_baseline = numpyro.sample("global_sigma_v_baseline", dist.HalfNormal(100))

        global_sigma_L_baseline = numpyro.sample("global_sigma_L_baseline", dist.HalfNormal(50))
        global_sigma_H_baseline = numpyro.sample("global_sigma_H_baseline", dist.HalfNormal(500))

        global_sigma_g_1_baseline = numpyro.sample("global_sigma_g_1_baseline", dist.HalfNormal(100))
        global_sigma_g_2_baseline = numpyro.sample("global_sigma_g_2_baseline", dist.HalfNormal(100))

        """ Baseline """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_baseline", n_baseline, dim=-2):
                """ Hyper-priors """
                mu_a_baseline = numpyro.sample(
                    "mu_a_baseline",
                    dist.TruncatedNormal(50, 50, low=0)
                )
                sigma_a_baseline = numpyro.sample(
                    "sigma_a_baseline",
                    dist.HalfNormal(50)
                )

                sigma_b_baseline = numpyro.sample("sigma_b_baseline", dist.HalfNormal(global_sigma_b_baseline))
                sigma_v_baseline = numpyro.sample("sigma_v_baseline", dist.HalfNormal(global_sigma_v_baseline))

                sigma_L_baseline = numpyro.sample("sigma_L_baseline", dist.HalfNormal(global_sigma_L_baseline))
                sigma_H_baseline = numpyro.sample("sigma_H_baseline", dist.HalfNormal(global_sigma_H_baseline))

                sigma_g_1_baseline = numpyro.sample("sigma_g_1_baseline", dist.HalfNormal(global_sigma_g_1_baseline))
                sigma_g_2_baseline = numpyro.sample("sigma_g_2_baseline", dist.HalfNormal(global_sigma_g_2_baseline))

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_baseline = numpyro.sample(
                        "a_baseline",
                        dist.TruncatedNormal(mu_a_baseline, sigma_a_baseline, low=0)
                    )

                    b_baseline = numpyro.sample("b_baseline", dist.HalfNormal(sigma_b_baseline))
                    v_baseline = numpyro.sample("v_baseline", dist.HalfNormal(sigma_v_baseline))

                    L_baseline = numpyro.sample("L_baseline", dist.HalfNormal(sigma_L_baseline))
                    H_baseline = numpyro.sample("H_baseline", dist.HalfNormal(sigma_H_baseline))

                    g_1_baseline = numpyro.sample("g_1_baseline", dist.Exponential(sigma_g_1_baseline))
                    g_2_baseline = numpyro.sample("g_2_baseline", dist.Exponential(sigma_g_2_baseline))

        """ Delta """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_delta", n_delta, dim=-2):
                mu_a_delta = numpyro.deterministic("mu_a_delta", self.mu_a_delta)
                sigma_a_delta = numpyro.deterministic("sigma_a_delta", self.sigma_a_delta)

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    a_delta = numpyro.sample("a_delta", dist.Normal(mu_a_delta, sigma_a_delta))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    a = numpyro.deterministic(
                        site.a,
                        jnp.concatenate([a_baseline, a_baseline + a_delta], axis=1)
                    )

                    b = numpyro.deterministic(
                        site.b,
                        jnp.concatenate([b_baseline, b_baseline], axis=1)
                    )
                    v = numpyro.deterministic(
                        site.v,
                        jnp.concatenate([v_baseline, v_baseline], axis=1)
                    )

                    L = numpyro.deterministic(
                        site.L,
                        jnp.concatenate([L_baseline, L_baseline], axis=1)
                    )
                    H = numpyro.deterministic(
                        site.H,
                        jnp.concatenate([H_baseline, H_baseline], axis=1)
                    )

                    g_1 = numpyro.deterministic(
                        site.g_1,
                        jnp.concatenate([g_1_baseline, g_1_baseline], axis=1)
                    )
                    g_2 = numpyro.deterministic(
                        site.g_2,
                        jnp.concatenate([g_2_baseline, g_2_baseline], axis=1)
                    )

                    # a = numpyro.deterministic(site.a, a_baseline)

                    # b = numpyro.deterministic(site.b, b_baseline)
                    # v = numpyro.deterministic(site.v, v_baseline)

                    # L = numpyro.deterministic(site.L, L_baseline)
                    # H = numpyro.deterministic(site.H, H_baseline)

                    # g_1 = numpyro.deterministic(site.g_1, g_1_baseline)
                    # g_2 = numpyro.deterministic(site.g_2, g_2_baseline)

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[subject, feature0]
            + jnp.maximum(
                0,
                -1
                + (H[subject, feature0] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[subject, feature0], v[subject, feature0]) - 1)
                    * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0])),
                    1 / v[subject, feature0]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[subject, feature0] + g_2[subject, feature0] * (1 / mu) ** 2
        )

        # """ Penalty """
        # penalty = (jnp.fabs(baseline + delta) - (baseline + delta))
        # numpyro.factor("penalty", -penalty)

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


class HierarchicalBayesianModel(Baseline):
    LINK = HBM

    def __init__(self, config: Config):
        super(HierarchicalBayesianModel, self).__init__(config=config)
        self.combination_columns = self.features + [self.subject]

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        n_baseline = 1
        n_delta = 1

        global_sigma_b_baseline = numpyro.sample("global_sigma_b_baseline", dist.HalfNormal(100))
        global_sigma_v_baseline = numpyro.sample("global_sigma_v_baseline", dist.HalfNormal(100))

        global_sigma_L_baseline = numpyro.sample("global_sigma_L_baseline", dist.HalfNormal(50))
        global_sigma_H_baseline = numpyro.sample("global_sigma_H_baseline", dist.HalfNormal(500))

        global_sigma_g_1_baseline = numpyro.sample("global_sigma_g_1_baseline", dist.HalfNormal(100))
        global_sigma_g_2_baseline = numpyro.sample("global_sigma_g_2_baseline", dist.HalfNormal(100))

        """ Baseline """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_baseline", n_baseline, dim=-2):
                """ Hyper-priors """
                mu_a_baseline = numpyro.sample(
                    "mu_a_baseline",
                    dist.TruncatedNormal(50, 50, low=0)
                )
                sigma_a_baseline = numpyro.sample(
                    "sigma_a_baseline",
                    dist.HalfNormal(50)
                )

                sigma_b_baseline = numpyro.sample("sigma_b_baseline", dist.HalfNormal(global_sigma_b_baseline))
                sigma_v_baseline = numpyro.sample("sigma_v_baseline", dist.HalfNormal(global_sigma_v_baseline))

                sigma_L_baseline = numpyro.sample("sigma_L_baseline", dist.HalfNormal(global_sigma_L_baseline))
                sigma_H_baseline = numpyro.sample("sigma_H_baseline", dist.HalfNormal(global_sigma_H_baseline))

                sigma_g_1_baseline = numpyro.sample("sigma_g_1_baseline", dist.HalfNormal(global_sigma_g_1_baseline))
                sigma_g_2_baseline = numpyro.sample("sigma_g_2_baseline", dist.HalfNormal(global_sigma_g_2_baseline))

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    """ Priors """
                    a_baseline = numpyro.sample(
                        "a_baseline",
                        dist.TruncatedNormal(mu_a_baseline, sigma_a_baseline, low=0)
                    )

                    b_baseline = numpyro.sample("b_baseline", dist.HalfNormal(sigma_b_baseline))
                    v_baseline = numpyro.sample("v_baseline", dist.HalfNormal(sigma_v_baseline))

                    L_baseline = numpyro.sample("L_baseline", dist.HalfNormal(sigma_L_baseline))
                    H_baseline = numpyro.sample("H_baseline", dist.HalfNormal(sigma_H_baseline))

                    g_1_baseline = numpyro.sample("g_1_baseline", dist.Exponential(sigma_g_1_baseline))
                    g_2_baseline = numpyro.sample("g_2_baseline", dist.Exponential(sigma_g_2_baseline))

        """ Delta """
        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_delta", n_delta, dim=-2):
                mu_a_delta = numpyro.sample("mu_a_delta", dist.Normal(0, 100))
                sigma_a_delta = numpyro.sample("sigma_a_delta", dist.HalfNormal(100))

                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    a_delta = numpyro.sample("a_delta", dist.Normal(mu_a_delta, sigma_a_delta))

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate("n_feature0", n_feature0, dim=-2):
                with numpyro.plate(site.n_subject, n_subject, dim=-3):
                    a = numpyro.deterministic(
                        site.a,
                        jnp.concatenate([a_baseline, a_baseline + a_delta], axis=1)
                    )

                    b = numpyro.deterministic(
                        site.b,
                        jnp.concatenate([b_baseline, b_baseline], axis=1)
                    )
                    v = numpyro.deterministic(
                        site.v,
                        jnp.concatenate([v_baseline, v_baseline], axis=1)
                    )

                    L = numpyro.deterministic(
                        site.L,
                        jnp.concatenate([L_baseline, L_baseline], axis=1)
                    )
                    H = numpyro.deterministic(
                        site.H,
                        jnp.concatenate([H_baseline, H_baseline], axis=1)
                    )

                    g_1 = numpyro.deterministic(
                        site.g_1,
                        jnp.concatenate([g_1_baseline, g_1_baseline], axis=1)
                    )
                    g_2 = numpyro.deterministic(
                        site.g_2,
                        jnp.concatenate([g_2_baseline, g_2_baseline], axis=1)
                    )

                    # a = numpyro.deterministic(site.a, a_baseline)

                    # b = numpyro.deterministic(site.b, b_baseline)
                    # v = numpyro.deterministic(site.v, v_baseline)

                    # L = numpyro.deterministic(site.L, L_baseline)
                    # H = numpyro.deterministic(site.H, H_baseline)

                    # g_1 = numpyro.deterministic(site.g_1, g_1_baseline)
                    # g_2 = numpyro.deterministic(site.g_2, g_2_baseline)

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[subject, feature0]
            + jnp.maximum(
                0,
                -1
                + (H[subject, feature0] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[subject, feature0], v[subject, feature0]) - 1)
                    * jnp.exp(-b[subject, feature0] * (intensity - a[subject, feature0])),
                    1 / v[subject, feature0]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[subject, feature0] + g_2[subject, feature0] * (1 / mu) ** 2
        )

        # """ Penalty """
        # penalty = (jnp.fabs(baseline + delta) - (baseline + delta))
        # numpyro.factor("penalty", -penalty)

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


In [ ]:
prefix = "power-analysis-parallel"

mu_a_delta = -1.5
sigma_a_delta = 1

prefix_2 = f"mu_a_delta_{mu_a_delta}_sigma_a_delta_{sigma_a_delta}"

root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/human/tms/mixed-effects.toml")

CONFIG = Config(toml_path=toml_path)

MODEL = Simulation(config=CONFIG, mu_a_delta=mu_a_delta, sigma_a_delta=sigma_a_delta)

src = "/home/vishu/data/hbmep-processed/human/tms/data.csv"
DF = pd.read_csv(src)

DF[MODEL.features[0]] = 0

DF[MODEL.response] = DF[MODEL.response] * 1000

DF, ENCODER_DICT = MODEL.load(df=DF)

dest = os.path.join(MODEL.build_dir, "inference.pkl")
with open(dest, "rb") as g:
    _, MCMC, POSTERIOR_SAMPLES = pickle.load(g)


2023-10-02 13:40:38,172 - hbmep.config - INFO - Verifying configuration ...
2023-10-02 13:40:38,172 - hbmep.config - INFO - Success!
2023-10-02 13:40:38,186 - hbmep.model.baseline - INFO - Initialized model with hierarchical_bayesian link
2023-10-02 13:40:38,193 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/tms/fit/mixed-effects
2023-10-02 13:40:38,193 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/human/tms/fit/mixed-effects
2023-10-02 13:40:38,194 - hbmep.dataset.core - INFO - Processing data ...
2023-10-02 13:40:38,194 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [ ]:
# prediction_df = DF.copy()
# posterior_predictive = MODEL.predict(df=prediction_df, posterior_samples=POSTERIOR_SAMPLES)
# obs = posterior_predictive[site.obs]

# dfs = []
# n_draws = 5

# for draw_ind in range(n_draws):
#     df = prediction_df.copy()
#     df[MODEL.response] = obs[draw_ind, ...]
#     dfs.append(df)

# dest = os.path.join(MODEL.build_dir, prefix, "simulation.pdf")
# _plot(self=MODEL, df=DF, dfs=dfs, destination_path=dest)


# prediction_df = DF.copy()
# prediction_df[MODEL.features[0]] = prediction_df[MODEL.features[0]] + 1
# prediction_df = pd.concat([DF, prediction_df], ignore_index=True).copy()

# posterior_predictive = MODEL.predict(df=prediction_df, posterior_samples=POSTERIOR_SAMPLES)
# obs = posterior_predictive[site.obs]

# dfs = []
# n_draws = 5

# for draw_ind in range(n_draws):
#     df = prediction_df.copy()
#     df[MODEL.response] = obs[draw_ind, ...]
#     dfs.append(df)

# dest = os.path.join(MODEL.build_dir, prefix, "simulation_intervention.pdf")
# _plot(self=MODEL, df=DF, dfs=dfs, destination_path=dest)

In [ ]:
POSTERIOR_SAMPLES.keys()

dict_keys(['H', 'H_baseline', 'L', 'L_baseline', 'a', 'a_baseline', 'b', 'b_baseline', 'g_1', 'g_1_baseline', 'g_2', 'g_2_baseline', 'global_sigma_H_baseline', 'global_sigma_L_baseline', 'global_sigma_b_baseline', 'global_sigma_g_1_baseline', 'global_sigma_g_2_baseline', 'global_sigma_v_baseline', 'mu_a_baseline', 'sigma_H_baseline', 'sigma_L_baseline', 'sigma_a_baseline', 'sigma_b_baseline', 'sigma_g_1_baseline', 'sigma_g_2_baseline', 'sigma_v_baseline', 'v', 'v_baseline', 'µ', 'β'])

In [ ]:
priors = {
    site.a, "a_baseline",
    site.b, "b_baseline",
    site.v, "v_baseline",
    site.L, "L_baseline",
    site.H, "H_baseline",
    site.g_1, "g_1_baseline",
    site.g_2, "g_2_baseline",
    site.mu, site.beta
}

POST = {u: v for u, v in POSTERIOR_SAMPLES.items() if u not in priors}

# prediction_df = DF.copy()
# posterior_predictive = MODEL.predict(df=prediction_df, posterior_samples=POST)

# obs = posterior_predictive[site.obs]

# dfs = []
# n_draws = 5

# for draw_ind in range(n_draws):
#     df = prediction_df.copy()
#     df[MODEL.response] = obs[draw_ind, ...]
#     dfs.append(df)

# dest = os.path.join(MODEL.build_dir, prefix, "simulation_new_participants.pdf")
# _plot(self=MODEL, df=DF, dfs=dfs, destination_path=dest)


In [ ]:
""" Experiment """
TOTAL_SUBJECTS = 200

PREDICTION_DF = \
    pd.DataFrame(np.arange(0, TOTAL_SUBJECTS, 1), columns=[MODEL.subject]) \
    .merge(
        pd.DataFrame(np.arange(0, 2, 1), columns=MODEL.features),
        how="cross"
    ) \
    .merge(
        pd.DataFrame([0, 100], columns=[MODEL.intensity]),
        how="cross"
    )
PREDICTION_DF = MODEL.make_prediction_dataset(df=PREDICTION_DF, num_points=60)

POSTERIOR_PREDICTIVE = MODEL.predict(df=PREDICTION_DF, posterior_samples=POST)
OBS = np.array(POSTERIOR_PREDICTIVE[site.obs])

N_space = [2, 4, 8, 16, 24]
# N_space = [2, 4, 8]

keys = jax.random.split(MODEL.rng_key, num=2)

n_draws = 50
# n_draws = 2

draws_space = \
    jax.random.choice(
        key=keys[0],
        a=np.arange(0, CONFIG.MCMC_PARAMS["num_chains"] * CONFIG.MCMC_PARAMS["num_samples"], 1),
        shape=(n_draws,),
        replace=False
    ) \
    .tolist()

n_repeats = 50
# n_repeats = 2

repeats_space = \
    jax.random.choice(
        key=keys[1],
        a=np.arange(0, n_repeats * 100, 1),
        shape=(n_repeats,),
        replace=False
    ) \
    .tolist()


2023-10-02 13:40:38,455 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.01 sec


2023-10-02 13:41:14,093 - hbmep.utils.utils - INFO - func:predict took: 35.64 sec


In [ ]:
def _process(N_counter, draw_counter, repeat_counter):
    N = N_space[N_counter]
    draw_ind = draws_space[draw_counter]
    seed = repeats_space[repeat_counter]

    N_dir, draw_dir, seed_dir = f"N_{N}", f"draw_{draw_ind}", f"seed_{seed}"

    subjects_ind = \
        jax.random.choice(
            key=jax.random.PRNGKey(seed),
            a=np.arange(0, TOTAL_SUBJECTS, 1),
            shape=(N,),
            replace=False
        ) \
        .tolist()

    ind = PREDICTION_DF[MODEL.subject].isin(subjects_ind)
    df = PREDICTION_DF[ind].reset_index(drop=True).copy()
    df[MODEL.response] = OBS[draw_ind, ...][ind, ...]

    """ Build model """
    config = Config(toml_path=toml_path)
    config.BUILD_DIR = os.path.join(CONFIG.BUILD_DIR, prefix, prefix_2, draw_dir, N_dir, seed_dir)
    model = HierarchicalBayesianModel(config=config)

    """ Load data """
    df, _ = model.load(df=df)

    """ Fit """
    _, posterior_samples = model.run_inference(df=df)

    """ Predict """
    prediction_df = model.make_prediction_dataset(df=df, num_points=100)
    ppd = model.predict(df=prediction_df, posterior_samples=posterior_samples)

    """ Plot """
    model.render_recruitment_curves(df=df, posterior_samples=posterior_samples, prediction_df=prediction_df, posterior_predictive=ppd)
    model.render_predictive_check(df=df, prediction_df=prediction_df, posterior_predictive=ppd)

    """ Power """
    mu_delta = np.array(posterior_samples["mu_a_delta"])

    dst = os.path.join(model.build_dir, "mu_a_delta.npy")
    np.save(dst, mu_delta)

    config, df, prediction_df, _,  = None, None, None, None
    model, posterior_samples = None, None
    ppd  = None
    mu_delta = None

    del config, df, prediction_df, _, model, posterior_samples, ppd, mu_delta
    gc.collect()
    return



In [ ]:
from joblib import Parallel, delayed


parallel = Parallel(n_jobs=-1)
parallel(
    delayed(_process)(N_counter, draw_counter, repeat_counter) \
    for draw_counter in range(n_draws) \
    for N_counter in range(len(N_space)) \
    for repeat_counter in range(n_repeats)
)

2023-10-02 13:41:16,433 - hbmep.config - INFO - Verifying configuration ...
2023-10-02 13:41:16,433 - hbmep.config - INFO - Verifying configuration ...
2023-10-02 13:41:16,434 - hbmep.config - INFO - Success!
2023-10-02 13:41:16,434 - hbmep.config - INFO - Success!
2023-10-02 13:41:16,434 - hbmep.model.baseline - INFO - Initialized model with hierarchical_bayesian link
2023-10-02 13:41:16,434 - hbmep.model.baseline - INFO - Initialized model with hierarchical_bayesian link
2023-10-02 13:41:16,434 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/tms/fit/mixed-effects/power-analysis-parallel/mu_a_delta_-1.5_sigma_a_delta_1/draw_1622/N_4/seed_197
2023-10-02 13:41:16,434 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/human/tms/fit/mixed-effects/power-analysis-parallel/mu_a_delta_-1.5_sigma_a_delta_1/draw_1622/N_2/seed_95
2023-10-02 13:41:16,434 - hbmep.dataset.core - INFO - Copied 

[None, None, None, None, None, None, None, None, None, None, None, None]